In [63]:
import numpy as np
import madmom
import pickle
from madmom.ml.nn import NeuralNetwork
from copy import deepcopy

In [88]:
with open('datasets/madmom_models-master/onsets/2013/onsets_cnn.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    base_model = u.load()

In [92]:
'''
 'bias',
 'pad',
 'reset',
 'stride',
 'weights']
'''
lay = base_model.layers[1]

linear_fn = base_model.layers[0].activation_fn

print(lay.bias.shape)
print(lay.weights.shape)
print(lay.activation_fn)
print(lay.pad)

(10,)
(3, 10, 7, 3)
<function linear at 0x7fb6f74108b0>
valid


In [90]:
#lay.bias = np.zeros((1))
#lay.weights = np.array([[1,-5],[-5,2]]).reshape((1,1,2,2))
lay.activation_fn = linear_fn

In [91]:

base_model.layers = [base_model.layers[0], lay]
with open('models/toy_model.pickle', 'wb') as file_pi:
    pickle.dump(base_model, file_pi)

In [80]:
nn = NeuralNetwork.load('models/toy_model.pickle')


In [81]:
act = nn(np.eye(3))

In [82]:
print(act)

[[[ 3.]
  [-5.]]

 [[-5.]
  [ 3.]]]
